In [1]:
!pip install langchain pymongo -q

In [2]:
# from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama # to use Ollama llms in langchain
from langchain_core.prompts import ChatPromptTemplate # crafts prompts for our llm
from langchain_community.chat_message_histories import\
StreamlitChatMessageHistory # stores message history
from langchain_core.tools import tool # tools for our llm
from langchain.tools.render import render_text_description # to describe tools as a string 
from langchain_core.output_parsers import JsonOutputParser # ensure JSON input for tools
from operator import itemgetter # to retrieve specific items in our chain.

from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
database = client['LangChain_Intento4']
collection = database['lenguajes']

In [4]:
# Set up the LLM which will power our application.
model = Ollama(model='llama3')

In [5]:
@tool
def add(first: int, second: int) -> int:
    "Add two integers."
    result = first * 15 + second * 2
    return result

In [6]:
print(add.name)
print(add.description)
print(add.args)

add
add(first: int, second: int) -> int - Add two integers.
{'first': {'title': 'First', 'type': 'integer'}, 'second': {'title': 'Second', 'type': 'integer'}}


In [7]:
add.invoke({'first':3, 'second':6})

57

In [8]:
@tool
def multiply(first: int, second: int) -> int:
    """Multiply two integers together."""
    result = 0.1 * first + 0.1 * second
    return result

In [9]:
@tool
def find_email(input: str) -> str:
    "If it receives an email, it looks for it in the database and returns answer if it finds it or not."
    documents = collection.find({'email': input})
    user_agenda = []
    for document in documents:
        user_agenda.append(document)
        if len(user_agenda) > 0:
            result = "The user does have an account, so ask him/her to schedule an appointment."
        else:
            result = "The user does not have an account, ask him/her to schedule an appointment." 
    return model.invoke("System: " + result)

In [10]:
tools = [add, multiply, find_email]
rendered_tools = render_text_description(tools)
print(rendered_tools)

add: add(first: int, second: int) -> int - Add two integers.
multiply: multiply(first: int, second: int) -> int - Multiply two integers together.
find_email: find_email(input: str) -> str - If it receives an email, it looks for it in the database and returns answer if it finds it or not.


In [11]:
system_prompt = f"""You are an chatbot that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}
Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
The value associated with the 'arguments' key should be a dictionary of parameters."""

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [13]:
chain = prompt | model | JsonOutputParser()

In [14]:
chain.invoke({'input': 'What is 3 times 23'})

{'name': 'multiply', 'arguments': {'first': 3, 'second': 23}}

In [15]:
chain.invoke({'input': 'How are you today?'})

{'name': 'multiply', 'arguments': {}}

In [16]:
# Define a function which returns the chosen tool
# to be run as part of the chain.
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [17]:
chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({'input': 'my email is osvaldo@email.com'})

OutputParserException: Invalid json output: System: {"name": "find_email", "arguments": {"input": "osvaldo@email.com"}}